<a target="_blank" href="https://colab.research.google.com/github/andrzejmizera/Multispectral-Edge-Filtered-Conditional-GAN/blob/main/cloudremoval_MEcGAN.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

**Remark:** If you find this case study code too complicated to start with, you may want to first have a look at this tutorial https://www.tensorflow.org/tutorials/generative/dcgan, which demonstrates how to generate images of handwritten digits using a Deep Convolutional Generative Adversarial Network (DCGAN).

# Download Dataset

The dataset can be downloaded with this link:

https://drive.google.com/file/d/1sqII7SSnbpfGRGYx4r4w170-eelnPAve/view?usp=sharing

Please upload it to your Google drive.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import zipfile
# Set path to the dataset.zip file placed on your Google drive. 
zf = zipfile.ZipFile('dataset.zip')
zf.extractall()

# Import Required Libraries

In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
    raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))
import numpy as np
import cv2
import progressbar
import datetime
import os
import time
import matplotlib

from datetime import datetime
from matplotlib import pyplot as plt
from IPython import display

%config IPCompleter.greedy=True

# Prepare Data Piplines

In [ ]:
dir_rgb = 'dataset/RGB'
dir_nir = 'dataset/NIR'
dir_clouds = 'dataset/clouds'

rgb_images = sorted(os.listdir(dir_rgb))
rgb_images.remove('.DS_Store')
cloud_images = sorted(os.listdir(dir_clouds))
cloud_images.remove('.DS_Store')
nir_images = sorted(os.listdir(dir_nir))
nir_images.remove('.DS_Store')

num_images = 528
train_num_images = 428

nir_cloud_intensity = 0.99

print('Prepare Training Dataset')
train_dataset_target_cloudfree_rgb = []
train_dataset_clouded_rgb = []
train_dataset_clouded_nir = []
pbar = progressbar.ProgressBar()
for i in pbar(range(train_num_images)):
    target_cloudfree_rgb = plt.imread(os.path.join(dir_rgb, rgb_images[i])).astype(np.float32)
    target_cloudfree_rgb = cv2.normalize(target_cloudfree_rgb, None, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)

    cloud = plt.imread(os.path.join(dir_clouds, cloud_images[i])).astype(np.float32)
    cloud = 255 * cloud
    cloud[:, :, 3] = np.clip(cloud[:, :, 3] + 75.0, 0, 255.0)
    alpha = cloud[:, :, 3] / 255.0
    alpha = np.broadcast_to(alpha[:, :, None], alpha.shape + (3,))

    # Merge RGB image with the selected cloud image.
    clouded_rgb = (1. - alpha) * target_cloudfree_rgb + alpha * np.ones([256, 256, 3])
    target_cloudfree_rgb = 2 * target_cloudfree_rgb - 1
    clouded_rgb = 2 * clouded_rgb - 1

    # Merge NIR image with the selected cloud image.    
    nir = plt.imread(os.path.join(dir_nir, nir_images[i])).astype(np.float32)
    nir = cv2.normalize(nir, None, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)
    nir_cloud = cloud[:, :, 3] / 255
    clouded_nir = (1. - nir_cloud_intensity * nir_cloud) * nir + nir_cloud_intensity * nir_cloud * np.ones([256, 256])
    clouded_nir = 2 * clouded_nir - 1

    train_dataset_target_cloudfree_rgb.append(target_cloudfree_rgb.astype(np.float32))
    train_dataset_clouded_rgb.append(clouded_rgb.astype(np.float32))
    train_dataset_clouded_nir.append(clouded_nir.astype(np.float32))

train_dataset_target_cloudfree_rgb = tf.data.Dataset.from_tensor_slices(train_dataset_target_cloudfree_rgb)
train_dataset_clouded_rgb = tf.data.Dataset.from_tensor_slices(train_dataset_clouded_rgb)
train_dataset_clouded_nir = tf.data.Dataset.from_tensor_slices(train_dataset_clouded_nir)
train_dataset = tf.data.Dataset.zip((train_dataset_clouded_rgb, train_dataset_target_cloudfree_rgb, train_dataset_clouded_nir))

print('Prepare Test Dataset')
test_dataset_target_cloudfree_rgb = []
test_dataset_clouded_rgb = []
test_dataset_clouded_nir = []
pbar = progressbar.ProgressBar()
for i in pbar(range(train_num_images, num_images)):
    target_cloudfree_rgb = plt.imread(os.path.join(dir_rgb, rgb_images[i])).astype(np.float32)
    target_cloudfree_rgb = cv2.normalize(target_cloudfree_rgb, None, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)

    cloud = plt.imread(os.path.join(dir_clouds, cloud_images[i])).astype(np.float32)
    cloud = 255 * cloud
    cloud[:, :, 3] = np.clip(cloud[:, :, 3] + 75.0, 0, 255.0)
    alpha = cloud[:, :, 3] / 255.0
    alpha = np.broadcast_to(alpha[:, :, None], alpha.shape + (3,))

    # Merge RGB image with the selected cloud image.
    clouded_rgb = (1. - alpha) * target_cloudfree_rgb + alpha * np.ones([256, 256, 3])
    target_cloudfree_rgb = 2 * target_cloudfree_rgb - 1
    clouded_rgb = 2 * clouded_rgb - 1

    # Merge NIR image with the selected cloud image.
    nir = plt.imread(os.path.join(dir_nir, nir_images[i])).astype(np.float32)
    nir = cv2.normalize(nir, None, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)
    nir_cloud = cloud[:, :, 3] / 255
    clouded_nir = (1. - nir_cloud_intensity * nir_cloud) * nir + nir_cloud_intensity * nir_cloud * np.ones([256, 256])
    clouded_nir = 2 * clouded_nir - 1

    test_dataset_target_cloudfree_rgb.append(target_cloudfree_rgb.astype(np.float32))
    test_dataset_clouded_rgb.append(clouded_rgb.astype(np.float32))
    test_dataset_clouded_nir.append(clouded_nir.astype(np.float32))

test_dataset_target_cloudfree_rgb = tf.data.Dataset.from_tensor_slices(test_dataset_target_cloudfree_rgb)
test_dataset_clouded_rgb = tf.data.Dataset.from_tensor_slices(test_dataset_clouded_rgb)
test_dataset_clouded_nir = tf.data.Dataset.from_tensor_slices(test_dataset_clouded_nir)
test_dataset = tf.data.Dataset.zip((test_dataset_clouded_rgb, test_dataset_target_cloudfree_rgb, test_dataset_clouded_nir))

# GAN Architecture

## Single Decoder and Encoder Layers

In [ ]:
# Single decoder layer 
def downsample(filters, size, apply_batchnorm=True):
    initializer = tf.random_normal_initializer(0., 0.02)
    result = tf.keras.Sequential()
    result.add(
        tf.keras.layers.Conv2D(filters, size, strides=2, padding='same',
                               kernel_initializer=initializer, use_bias=False))
    if apply_batchnorm:
        result.add(tf.keras.layers.BatchNormalization())
        result.add(tf.keras.layers.LeakyReLU())
    return result

# Single encoder layer
def upsample(filters, size, apply_dropout=False):
    initializer = tf.random_normal_initializer(0., 0.02)
    result = tf.keras.Sequential()
    result.add(tf.keras.layers.Conv2DTranspose(filters, size, strides=2,
                                               padding='same',
                                               kernel_initializer=initializer,
                                               use_bias=False))
    result.add(tf.keras.layers.BatchNormalization())
    if apply_dropout:
        result.add(tf.keras.layers.Dropout(0.5))

    result.add(tf.keras.layers.ReLU())
    return result

## Generator Architecture and Loss

In [ ]:
OUTPUT_CHANNELS = 3
def Generator():
    clouded_rgb = tf.keras.layers.Input(shape=[256, 256, 3], name='clouded_rgb')
    clouded_nir = tf.keras.layers.Input(shape=[256, 256, 1], name='clouded_nir')
    down_stack = [
        downsample(64, 4, apply_batchnorm=False),
        downsample(128, 4),
        downsample(256, 4),
        downsample(512, 4),
        downsample(512, 4),
        downsample(512, 4),
        downsample(512, 4),
        downsample(512, 4),
    ]
    up_stack = [
        upsample(512, 4, apply_dropout=True),
        upsample(512, 4, apply_dropout=True),
        upsample(512, 4, apply_dropout=True),
        upsample(512, 4),
        upsample(256, 4),
        upsample(128, 4),
        upsample(64, 4),
    ]
    initializer = tf.random_normal_initializer(0., 0.02)
    last = tf.keras.layers.Conv2DTranspose(OUTPUT_CHANNELS, 4,
                                           strides=2,
                                           padding='same',
                                           kernel_initializer=initializer,
                                           activation='tanh')  # (bs, 256, 256, 3)
    x = tf.keras.layers.concatenate([clouded_rgb, clouded_nir])
    # Downsampling through the model
    skips = []
    for down in down_stack:
        x = down(x)
        skips.append(x)
    skips = reversed(skips[:-1])
    # Upsampling and establishing the skip connections
    for up, skip in zip(up_stack, skips):
        x = up(x)
        x = tf.keras.layers.Concatenate()([x, skip])
    x = last(x)
    return tf.keras.Model(inputs=[clouded_rgb, clouded_nir], outputs=x)

LAMBDA = 1

loss_object = tf.keras.losses.BinaryCrossentropy(from_logits=False)

def generator_loss(disc_generated_output, generated_cloudfree_rgb, target_cloudfree_rgb):
    gan_loss = loss_object(tf.ones_like(disc_generated_output), disc_generated_output)
    # mean absolute error
    l1_loss = tf.reduce_mean(tf.abs(target_cloudfree_rgb - generated_cloudfree_rgb))
    total_gen_loss = gan_loss + LAMBDA * l1_loss
    return total_gen_loss, gan_loss, l1_loss

## Discriminator Architecture and Loss

In [ ]:
def Discriminator():
    initializer = tf.random_normal_initializer(0., 0.02)
    # edge_clouded_nir = tf.keras.layers.Input(shape=[256, 256, 1], name='edge_clouded_nir')
    cloud_free_rgb = tf.keras.layers.Input(shape=[256, 256, 3], name='cloud_free_rgb')
    edge_cloud_free_rgb = tf.keras.layers.Input(shape=[256, 256, 1], name='edge_cloud_free_rgb')
    # gray_edge_clouded_rgb = tf.keras.layers.Input(shape=[256, 256, 1], name='gray_edge_clouded_rgb')
    x = tf.keras.layers.concatenate([cloud_free_rgb, edge_cloud_free_rgb])
    down1 = downsample(64, 3, False)(x)
    down2 = downsample(128, 3)(down1)
    down3 = downsample(128, 3)(down2)
    down4 = downsample(256, 3)(down3)
    down5 = downsample(256, 3)(down4)
    down6 = downsample(512, 3)(down5)
    down7 = downsample(512, 3)(down6)
    zero_pad1 = tf.keras.layers.ZeroPadding2D()(down7)
    conv = tf.keras.layers.Conv2D(512, 3, strides=1,
                                  kernel_initializer=initializer,
                                  use_bias=False)(zero_pad1)
    batchnorm1 = tf.keras.layers.BatchNormalization()(conv)
    leaky_relu = tf.keras.layers.LeakyReLU()(batchnorm1)
    zero_pad2 = tf.keras.layers.ZeroPadding2D()(leaky_relu)
    layer10 = tf.keras.layers.Conv2D(1, 3, strides=1,
                                     kernel_initializer=initializer)(zero_pad2)
    flatten_layer = tf.keras.layers.Flatten()(layer10)
    dense = tf.keras.layers.Dense(1)(flatten_layer)
    return tf.keras.Model(inputs=[cloud_free_rgb, edge_cloud_free_rgb], outputs=tf.keras.activations.sigmoid(dense))

#loss_object = tf.keras.losses.BinaryCrossentropy(from_logits=True)

def discriminator_loss(disc_real_output, disc_generated_output):
    real_loss = loss_object(tf.ones_like(disc_real_output), disc_real_output)
    generated_loss = loss_object(tf.zeros_like(disc_generated_output), disc_generated_output)
    total_disc_loss = real_loss + generated_loss
    return total_disc_loss

# Visualize Images

In [ ]:
def display_images(generated_cloudfree_rgb, test_input, target_cloudfree_rgb):
    plt.figure(figsize=(15,15))
    display_list = [test_input[0][0], test_input[1][0,:,:,0], target_cloudfree_rgb[0], generated_cloudfree_rgb[0]]
    title = ['Clouded RGB', 'Clouded NIR', 'Target Cloud-free RGB', 'Generated Cloud-free RGB']
    for i in range(4):
        plt.subplot(1, 4, i+1)
        plt.title(title[i])
        # getting the pixel values between [0, 1] to plot it.
        if i==1:
            plt.imshow((display_list[i].numpy() + 1)/2.0, cmap='gray')
        else:
            plt.imshow((display_list[i].numpy() + 1)/2.0)
        plt.axis('off')
    plt.show()

# Training

In [ ]:
# Notice the use of `tf.function`
# This annotation causes the function to be "compiled".
@tf.function
def train_step(clouded_rgb, target_cloudfree_rgb, clouded_nir, epoch):
    
    # To better understand gradient tapes, please look at
    #   https://www.tensorflow.org/guide/autodiff
    # and 
    #   https://www.tensorflow.org/api_docs/python/tf/GradientTape.
    # We use two independent gradient tapes, since calling the gradient() method destroys the tape.
    # We need to call the method first to differentiate the losses with respect to the generator parameters
    # and the discriminator parameters.
    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
        
        # Generate cloud-free RGB image from clouded RGB image and its corresponding NIR image
        # (notice that batch dimension is added to the images, and the colour dimension to the NIR image)
        # training parameter: Boolean or boolean scalar tensor, indicating whether to run the Network in
        #                     training mode or inference mode. 
        generated_cloudfree_rgb = generator([clouded_rgb[None, :, :, :], clouded_nir[None, :, :, None]], training=True)

        # Grayscaling the generated cloud-free RGB image
        gray_generated_cloudfree_rgb = tf.image.rgb_to_grayscale(generated_cloudfree_rgb)
        # Applying edge detection on the grayscaled generated cloud-free RGB image (Sobel method)
        sobel = tf.image.sobel_edges(gray_generated_cloudfree_rgb)
        gray_edge_generated_cloudfree_rgb = sobel[:, :, :, :, 0] # sobel in y-direction

        # Grayscaling the taget cloud-free RGB image (batch dimension needs to be added to apply
        # the tf.image.rgb_to_grayscale function)
        gray_target_cloudfree_rgb = tf.image.rgb_to_grayscale(target_cloudfree_rgb[None, :, :, :])
        # Applying edge detection on the grayscaled generated cloud-free RGB image (Sobel method)
        gray_edge_target_cloudfree_rgb = tf.image.sobel_edges(gray_target_cloudfree_rgb)
        gray_edge_target_cloudfree_rgb = gray_edge_target_cloudfree_rgb[:, :, :, :, 0] # sobel in y-direction

        # Apply the discriminator on the target cloud-free RGB images and their edges
        # (notice that batch dimension is added to the target cloud-free RGB images)
        disc_real_output = discriminator([target_cloudfree_rgb[None, :, :, :], gray_edge_target_cloudfree_rgb], training=True)
        # Apply the discriminator on the generated cloud-free RGB iamges and their edges
        disc_generated_output = discriminator([generated_cloudfree_rgb, gray_edge_generated_cloudfree_rgb], training=True)

        # Compute the generator loss 
        gen_total_loss, gen_gan_loss, gen_l1_loss = generator_loss(disc_generated_output, generated_cloudfree_rgb,
                                                                   target_cloudfree_rgb)
        # Compute the generator loss for the discriminator output for the real images and generated images
        disc_loss = discriminator_loss(disc_real_output, disc_generated_output)

    # Compute the gradients for the generator
    generator_gradients = gen_tape.gradient(gen_total_loss, generator.trainable_variables)
    # Compute the gradients for the discriminator
    discriminator_gradients = disc_tape.gradient(disc_loss, discriminator.trainable_variables)
    
    # Update the parameters of the generator
    generator_optimizer.apply_gradients(zip(generator_gradients, generator.trainable_variables))
    # Update the parameters of the discriminator
    discriminator_optimizer.apply_gradients(zip(discriminator_gradients, discriminator.trainable_variables))

    # Logging loss values
    with summary_writer.as_default():
        tf.summary.scalar('gen_total_loss', gen_total_loss, step=epoch)
        tf.summary.scalar('gen_gan_loss', gen_gan_loss, step=epoch)
        tf.summary.scalar('gen_l1_loss', gen_l1_loss, step=epoch)
        tf.summary.scalar('disc_loss', disc_loss, step=epoch)

dir_generated_cloud_free_images = 'results/' + 'generated_cloud_free_images_' + str(nir_cloud_intensity * 100) + 'eta_NIR'
try:
    os.makedirs(dir_generated_cloud_free_images)
except OSError:
    print("Directory %s already exists" % dir_generated_cloud_free_images)

def fit(train_ds, test_ds, start_from_epoch):
    len_epochs = 10 ** (len(str(EPOCHS)))
    for epoch in range(start_from_epoch - 1, EPOCHS):
        print("Epoch: ", epoch + 1)
        pbar = progressbar.ProgressBar(maxval=train_num_images)
        pbar.start()
        idx = 1
        for _, all_images_in_batch in train_ds.enumerate():
            train_step(all_images_in_batch[0], all_images_in_batch[1], all_images_in_batch[2], epoch)
            pbar.update(idx)
            idx = idx + 1
        pbar.finish()

        # saving generated cloud-free rgb images in every 10 epochs
        if (epoch + 1) % 100 == 0:
            id_im = 1
            for example_clouded_rgb_image, example_target_cloudfree_rgb_image, example_clouded_nir in test_ds.take(num_images - train_num_images):
                generated_cloud_free_image = generator([example_clouded_rgb_image[None, :, :, :], example_clouded_nir[None, :, :, None]], training=True)                
                matplotlib.image.imsave(dir_generated_cloud_free_images + '/' + 'generated_cloud_free_rgb' + str(len_epochs + epoch + 1)[1:] + '_' + str(10 * (num_images - train_num_images) + id_im)[1:] + '.png', (1 + generated_cloud_free_image[0].numpy()) / 2)
                # visualize generated cloud-free rgb images
                display_images(generated_cloud_free_image, [example_clouded_rgb_image[None,:,:,:], example_clouded_nir[None,:,:,None]], example_target_cloudfree_rgb_image[None,:,:,:])
                id_im = id_im + 1        
        
        # saving (checkpoint) the model in every 20 epochs
        if (epoch + 1) % 10 == 0:
            checkpoint.save(file_prefix=checkpoint_prefix)

    checkpoint.save(file_prefix=checkpoint_prefix)

# Fit Data

In [ ]:
generator = Generator()
discriminator = Discriminator()
generator_optimizer = tf.keras.optimizers.Adam(2e-4, beta_1=0.5)
discriminator_optimizer = tf.keras.optimizers.Adam(2e-4, beta_1=0.5)

In [ ]:
log_dir = 'logs/'
summary_writer = tf.summary.create_file_writer(log_dir + 'fit/' + datetime.now().strftime("%Y%m%d-%H%M%S"))

checkpoint_dir = './training_checkpoints'
try:
    os.mkdir(checkpoint_dir)
except OSError:
    print("Directory %s already exists" % checkpoint_dir)

checkpoint_prefix = os.path.join(checkpoint_dir, 'ckpt')

checkpoint = tf.train.Checkpoint(generator_optimizer=generator_optimizer,
                              discriminator_optimizer=discriminator_optimizer,
                              generator=generator,
                              discriminator=discriminator)

restore_checkpoint = False # To restore last checkpoint, set this to True 
if restore_checkpoint is False:
    checkpoint_path = checkpoint_dir + '/training/cp-{epoch:04d}.ckpt'
else:
    checkpoint.restore(tf.train.latest_checkpoint(checkpoint_dir))
    
EPOCHS = 500
start_from_epoch = 1
fit(train_dataset, test_dataset, start_from_epoch)

# Use Saved Checkpoint to Generate Cloud-free RGB Images

In [ ]:
generator = Generator()
discriminator = Discriminator()
generator_optimizer = tf.keras.optimizers.Adam(2e-4, beta_1=0.5)
discriminator_optimizer = tf.keras.optimizers.Adam(2e-4, beta_1=0.5)

checkpoint_dir = './training_checkpoints'
try:
    os.mkdir(checkpoint_dir)
except OSError:
    print("Directory %s already exists" % checkpoint_dir)

checkpoint_prefix = os.path.join(checkpoint_dir, 'ckpt')

checkpoint = tf.train.Checkpoint(generator_optimizer=generator_optimizer,
                              discriminator_optimizer=discriminator_optimizer,
                              generator=generator,
                              discriminator=discriminator)

checkpoint.restore(tf.train.latest_checkpoint(checkpoint_dir))


for example_clouded_rgb_image, example_target_cloudfree_rgb_image, example_clouded_nir in test_dataset.take(num_images - train_num_images):
  generated_cloud_free_image = generator([example_clouded_rgb_image[None, :, :, :], example_clouded_nir[None, :, :, None]], training=True)                    
  # visualize generated cloud-free rgb images
  display_images(generated_cloud_free_image, [example_clouded_rgb_image[None,:,:,:], example_clouded_nir[None,:,:,None]], example_target_cloudfree_rgb_image[None,:,:,:])    

In [ ]:
def save_images(generated_images, save_dir):
    try:
        os.mkdir(save_dir)
    except OSError:
        print("Directory %s already exists" % save_dir)

    for i, generated_image in enumerate(generated_images):
        save_path = os.path.join(save_dir, f'image_{i}.png')
        plt.imsave(save_path, (generated_image.numpy() + 1) / 2.0)
        
save_dir = './generated_images'
save_images(generated_cloud_free_image, save_dir)
